In [1]:
# Manipulate the file system
import os
import shutil

# Copy dictionaries
import copy

# Regular expressions
import re

# Display errors in realtime
import ipywidgets as widgets

# work with dates
import datetime
import arrow
import time

# For scrubbing PII
import hashlib

# Convert stored string representation of a list to a list
import ast

# Recurse through a directory tree and return file names with glob
import glob

# Decode and re-encode mangled Arabic file names
import codecs

# Connect to a SQLite database in a lazy manner.
import sqlalchemy
import dataset

# Enables opening and reading of Excel files
import openpyxl

# Translating variables, sheet names, and workbook names from Arabic
# This is NOT free to use.
from google.cloud import translate

# Set the environment variable for the Google Service Account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\clay\\Documents\\fxb-lcs-2b24f4f8a73a.json'

In [2]:
#If there's an existing db for this sheet, delete it
#so that we can copy from the template for a fresh start

try:
    os.remove("sams_data_phase07.sqlite")
    print("Removed template clone sams_data_phase07.sqlite")
except:
    pass

try:
    # Try to preserve a copy in case there is a problem and it has to be restored
    shutil.copy2("sams_data_phase07_template.sqlite","sams_data_phase07.sqlite")
    
    print("Created database from template: sams_data_phase07.sqlite")
except:
    pass

Removed template clone sams_data_phase07.sqlite
Created database from template: sams_data_phase07.sqlite


In [3]:
db = dataset.connect("sqlite:///sams_data_phase07.sqlite")

In [4]:
tab_arabic = db['arabic_values']

In [5]:
tab_tokens = db['arabic_tokens']
tab_arabic_values_tokens = db['arabic_values_tokens']

In [6]:
good_char = 'ا'

In [7]:
bad_char1 = 'أ'

In [8]:
bad_char2 = 'إ'

In [9]:
bad_char3 = 'آ'

In [10]:
delimiters = "(", ")", ",", "/", ".", "1","2","3","4","5","6","7","8","9","0","+","_","-","\\","=","ـ"
regexpattern = "|".join(map(re.escape,delimiters))

In [11]:
token_lookup = {}

for rec in tab_arabic.find():
    tokens = rec['arabic']
    tokens = tokens.replace(bad_char1,good_char)
    tokens = tokens.replace(bad_char2,good_char)
    tokens = tokens.replace(bad_char3,good_char)
    
    tokens = [t.strip() for t in re.split(regexpattern,tokens) if t.strip() != '']
    
    
#     tokens = tokens.replace("("," ")
#     tokens = tokens.replace(")"," ")
#     tokens = tokens.replace(" - "," ")
#     tokens = tokens.replace("-"," ")
#     tokens = tokens.replace("+"," ")

#     tokens = tokens.replace('-',' ').replace('-',' ')
#     tokens = tokens.replace(',',' ').replace(',',' ')
#     tokens = tokens.replace('_',' ').replace('_',' ')
#     tokens = tokens.replace('+',' ').replace('+',' ')
#     tokens = tokens.replace('/',' ').replace('/',' ')
#     tokens = tokens.replace(')',' ').replace(')',' ')
#     tokens = tokens.replace('(',' ').replace('(',' ')
#     tokens = tokens.replace('.',' ').replace('.',' ')
#     tokens = tokens.replace('\\',' ').replace('ـ',' ').replace('=',' ')
#     tokens = tokens.replace('0',' ').replace('1',' ').replace('2',' ').replace('3',' ')
#     tokens = tokens.replace('4',' ').replace('5',' ').replace('6',' ').replace('7',' ')
#     tokens = tokens.replace('8',' ').replace('9',' ')
#     tokens = tokens.strip().split()
    
    for token in tokens:
        if token not in token_lookup.keys():
            token_lookup[token] = {"token_id":None, "arabic_values_id":[]}
        token_lookup[token]["arabic_values_id"].append(rec['id'])
        

In [12]:
len(token_lookup.keys())

153460

In [13]:
tokens_to_insert = []
for token in token_lookup.keys():
    tokens_to_insert.append({"token":token,"translation":None})

In [14]:
tab_tokens.insert_many(tokens_to_insert)

In [15]:
for rec in tab_tokens.find():
    token_lookup[rec['token']]["token_id"] = rec['id']

In [16]:
token_lookup[list(token_lookup.keys())[0]]

{'arabic_values_id': [1, 12001, 12002, 15950], 'token_id': 1}

In [17]:
join_records = []
for token in token_lookup.keys():
    for orig_id in token_lookup[token]["arabic_values_id"]:
        join_records.append({"token_id":token_lookup[token]["token_id"],"arabic_values_id":orig_id})

In [18]:
len(join_records)

728875

In [19]:
tab_arabic_values_tokens.insert_many(join_records)

In [21]:
# Copy the database over as the template for the next file.
# This Notebook did not include manual editing of the data.

# Do not rerun this cell!
shutil.copy2('sams_data_phase07.sqlite','sams_data_phase08_template.sqlite')

'sams_data_phase08_template.sqlite'